In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

#Download Dataset

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
fid = drive.ListFile({'q':"title='Dataset.zip'"}).GetList()[0]['id']
f = drive.CreateFile({'id': fid})
f.GetContentFile('Dataset.zip')

In [ ]:
f.keys()

dict_keys(['id', 'kind', 'etag', 'selfLink', 'webContentLink', 'alternateLink', 'embedLink', 'iconLink', 'title', 'mimeType', 'labels', 'copyRequiresWriterPermission', 'createdDate', 'modifiedDate', 'modifiedByMeDate', 'lastViewedByMeDate', 'markedViewedByMeDate', 'version', 'parents', 'downloadUrl', 'userPermission', 'originalFilename', 'fileExtension', 'md5Checksum', 'fileSize', 'quotaBytesUsed', 'ownerNames', 'owners', 'lastModifyingUserName', 'lastModifyingUser', 'capabilities', 'editable', 'copyable', 'writersCanShare', 'shared', 'explicitlyTrashed', 'appDataContents', 'headRevisionId', 'spaces'])

In [ ]:
PATH = '/content/Dataset'

In [ ]:
!unzip Dataset.zip

เอาต์พุตของการสตรีมมีการตัดเหลือเพียง 5000 บรรทัดสุดท้าย
  inflating: Dataset_X_6/RTD/RDT_X_1_4_0233.png  
  inflating: Dataset_X_6/RTD/RDT_X_1_4_0234.png  
  inflating: Dataset_X_6/RTD/RDT_X_1_4_0235.png  
  inflating: Dataset_X_6/RTD/RDT_X_1_4_0236.png  
  inflating: Dataset_X_6/RTD/RDT_X_1_4_0237.png  
  inflating: Dataset_X_6/RTD/RDT_X_1_4_0238.png  
  inflating: Dataset_X_6/RTD/RDT_X_1_4_0239.png  
  inflating: Dataset_X_6/RTD/RDT_X_1_4_0240.png  
  inflating: Dataset_X_6/RTD/RDT_X_1_4_0241.png  
  inflating: Dataset_X_6/RTD/RDT_X_1_4_0242.png  
  inflating: Dataset_X_6/RTD/RDT_X_1_4_0243.png  
  inflating: Dataset_X_6/RTD/RDT_X_1_4_0244.png  
  inflating: Dataset_X_6/RTD/RDT_X_1_4_0245.png  
  inflating: Dataset_X_6/RTD/RDT_X_1_4_0246.png  
  inflating: Dataset_X_6/RTD/RDT_X_1_4_0247.png  
  inflating: Dataset_X_6/RTD/RDT_X_1_4_0248.png  
  inflating: Dataset_X_6/RTD/RDT_X_1_4_0249.png  
  inflating: Dataset_X_6/RTD/RDT_X_1_4_0250.png  
  inflating: Dataset_X_6/RTD/RDT_X_1_4_0251

In [ ]:
os.listdir(PATH)

['NBSD', 'HL', 'RTD', 'BD', 'BSD', 'BLBD']

In [ ]:
train_dir = os.path.join(PATH)

In [ ]:
train_HL_dir = os.path.join(train_dir,
                               'HL')
train_BD_dir = os.path.join(train_dir,
                                'BD')
train_BLBD_dir = os.path.join(train_dir,
                                   'BLBD')
train_BSD_dir = os.path.join(train_dir,
                               'BSD')
train_RTD_dir = os.path.join(train_dir,
                                'RTD')
train_NBSD_dir = os.path.join(train_dir,
                                   'NBSD')

In [ ]:
# S=[train_HL_dir
# ,train_BD_dir
# ,train_BLBD_dir
# ,train_BSD_dir
# ,train_RTD_dir
# ,train_NBSD_dir
#  ]
# j=1
# Sum=0
# for i in S:
#   Sum=Sum+len(os.listdir(i))
#   print('t',j,':',len(os.listdir(i)))
#   j=j+1

#Preparing the data

In [ ]:
train_HL_dir

'/content/Dataset_X_6/HL'

In [ ]:
#https://machinelearningmastery.com/image-augmentation-deep-learning-keras/

In [ ]:
IMG_HEIGHT = 224
IMG_WIDTH = 224
batch_size = 256

In [ ]:
image_gen_train = ImageDataGenerator(
                    rescale=1./255
                    ,validation_split=0.2
                    )

In [ ]:
train_data_gen = image_gen_train.flow_from_directory(batch_size=batch_size,
                                                     directory=train_dir,
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     subset="training",
                                                     class_mode='categorical')
train_x_scaled, train_images_y_encoded = next(train_data_gen)

Found 43470 images belonging to 6 classes.


In [ ]:
validation_data_gen = image_gen_train.flow_from_directory(batch_size=batch_size,
                                                     directory=train_dir,
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     subset="validation",
                                                     class_mode='categorical')
val_x_scaled, val_images_y_encoded = next(validation_data_gen)

Found 10865 images belonging to 6 classes.


In [ ]:
# sample_training_images, _ = next(train_data_gen)

In [ ]:
# # This function will plot images in the form of a grid with 1 row and 5 columns where images are placed in each column.
# def plotImages(images_arr):
#     fig, axes = plt.subplots(1, 5, figsize=(20,20))
#     axes = axes.flatten()
#     for img, ax in zip( images_arr, axes):
#         ax.imshow(img)
#         ax.axis('off')
#     plt.tight_layout()
#     plt.show()

In [ ]:
# plotImages(sample_training_images[:5])

#Training the model

#Classify ImageNet classes with Model
* https://keras.io/api/applications/

In [ ]:
from keras.preprocessing import image

# import model
from tensorflow.keras.applications import Xception
from tensorflow.keras.applications import VGG16, VGG19
from tensorflow.keras.applications import ResNet50,ResNet101, ResNet152
from tensorflow.keras.applications import ResNet50V2,ResNet101V2, ResNet152V2
from tensorflow.keras.applications import InceptionV3, InceptionResNetV2
from tensorflow.keras.applications import MobileNet, MobileNetV2
from tensorflow.keras.applications import DenseNet121, DenseNet169, DenseNet201
from tensorflow.keras.applications import NASNetLarge, NASNetMobile
from tensorflow.keras.applications import EfficientNetB0, EfficientNetB2, EfficientNetB3
from tensorflow.keras.applications import EfficientNetB4, EfficientNetB5, EfficientNetB6
from tensorflow.keras.applications import EfficientNetB7,EfficientNetB1

#Checkpoint
from tensorflow.keras.callbacks import ModelCheckpoint

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input
from tensorflow.keras.optimizers import Adam
import numpy as np
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split

In [ ]:
def Train_Model():
  NameModel=[Xception,VGG16,VGG19,ResNet50,ResNet50V2,ResNet101,
     ResNet101V2,ResNet152,ResNet152V2,InceptionV3,InceptionResNetV2,
     MobileNet,MobileNetV2,DenseNet121,DenseNet169,DenseNet201,
     NASNetMobile,NASNetLarge ,EfficientNetB0 ,EfficientNetB1 ,
     EfficientNetB1 ,EfficientNetB2 ,EfficientNetB3 ,EfficientNetB4 ,
     EfficientNetB5,EfficientNetB6 ,EfficientNetB7]
  TextModel=["Xception","VGG16","VGG19","ResNet50","ResNet50V2","ResNet101",
     "ResNet101V2","ResNet152","ResNet152V2","InceptionV3","InceptionResNetV2",
     "MobileNet","MobileNetV2","DenseNet121","DenseNet169","DenseNet201",
     "NASNetMobile","NASNetLarge" ,"EfficientNetB0" ,"EfficientNetB1" ,
     "EfficientNetB1" ,"EfficientNetB2" ,"EfficientNetB3" ,"EfficientNetB4" ,
     "EfficientNetB5","EfficientNetB6" ,"EfficientNetB7"]

  Text="\
    Please enter the model you want to run \n\
     1.) Xception please enter 1 \n\
     2.) VGG16 please enter 2 \n\
     3.) VGG19 please enter 3 \n\
     4.) ResNet50 please enter 4 \n\
     5.) ResNet50V2 please enter 5 \n\
     6.) ResNet101 please enter 6 \n\
     7.) ResNet101V2 please enter 7 \n\
     8.) ResNet152 please enter 8 \n\
     9.) ResNet152V2 please enter 9 \n\
     10.) InceptionV3 please enter 10 \n\
     11.) InceptionResNetV2 please enter 11 \n\
     12.) MobileNet please enter 12 \n\
     13.) MobileNetV2 please enter 13 \n\
     14.) DenseNet121 please enter 14 \n\
     15.) DenseNet169 please enter 15 \n\
     16.) DenseNet201 please enter 16 \n\
     17.) NASNetMobile please enter 17 \n\
     18.) NASNetLarge please enter 18 \n\
     19.) EfficientNetB0 please enter 19 \n\
     20.) EfficientNetB1 please enter 20 \n\
     21.) EfficientNetB1 please enter 21 \n\
     22.) EfficientNetB2 please enter 22 \n\
     23.) EfficientNetB3 please enter 23 \n\
     24.) EfficientNetB4 please enter 24 \n\
     25.) EfficientNetB5 please enter 25 \n\
     26.) EfficientNetB6 please enter 26 \n\
     27.) EfficientNetB7 please enter 27 \n\
"
  choice = int(input(Text))

  # while (choice not in(1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,
  #                     19,20,21,22,23,24,25,26,27)):
  #       print("Invalid input, please try again")
  #       choice = input(Text)

  # Checkpoint In the /output folder
  filepath ='/content/drive/My Drive/Project_Pound/'+TextModel[choice-1]+'.hdf5'
  # Keep only a single checkpoint, the best over test accuracy.
  checkpoint =ModelCheckpoint(filepath, monitor = 'val_loss', verbose = 1,
    save_weights_only=True,
    save_best_only=True, 
    mode='max'
    )
  
  #Model
  num_classes = 6
  base_model = NameModel[choice-1]

  base_model = base_model(weights="imagenet", include_top=False)
  x = base_model.output
  x = GlobalAveragePooling2D()(x)
  x = Dense(256, activation='relu')(x)
  predictions = Dense(num_classes, activation='softmax')(x)
  model = Model(inputs=base_model.input, outputs=predictions)
  for layer in base_model.layers:
      layer.trainable = False

  model.compile(loss='categorical_crossentropy',
                optimizer=Adam(lr=0.0001),
                metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy( k=5, name='top_5_accuracy', dtype=None)])
  return model,checkpoint

In [ ]:
model,checkpoint=Train_Model()

    Please enter the model you want to run 
     1.) Xception please enter 1 
     2.) VGG16 please enter 2 
     3.) VGG19 please enter 3 
     4.) ResNet50 please enter 4 
     5.) ResNet50V2 please enter 5 
     6.) ResNet101 please enter 6 
     7.) ResNet101V2 please enter 7 
     8.) ResNet152 please enter 8 
     9.) ResNet152V2 please enter 9 
     10.) InceptionV3 please enter 10 
     11.) InceptionResNetV2 please enter 11 
     12.) MobileNet please enter 12 
     13.) MobileNetV2 please enter 13 
     14.) DenseNet121 please enter 14 
     15.) DenseNet169 please enter 15 
     16.) DenseNet201 please enter 16 
     17.) NASNetMobile please enter 17 
     18.) NASNetLarge please enter 18 
     19.) EfficientNetB0 please enter 19 
     20.) EfficientNetB1 please enter 20 
     21.) EfficientNetB1 please enter 21 
     22.) EfficientNetB2 please enter 22 
     23.) EfficientNetB3 please enter 23 
     24.) EfficientNetB4 please enter 24 
     25.) EfficientNetB5 please enter 

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning:

The `lr` argument is deprecated, use `learning_rate` instead.



In [ ]:
batch_size = 256
epochs=30

In [ ]:
# model.summary()

In [ ]:
history = model.fit(train_x_scaled, train_images_y_encoded, batch_size=batch_size, epochs=epochs, 
                    shuffle=True
                    ,validation_split=0.1
                    ,verbose=1
                    ,callbacks=[checkpoint],
                    )

Epoch 1/30
1/1 [==============================] - 4s 4s/step - loss: 1.8163 - accuracy: 0.1783 - top_5_accuracy: 0.8739 - val_loss: 1.7804 - val_accuracy: 0.3077 - val_top_5_accuracy: 0.8846

Epoch 00001: val_loss improved from -inf to 1.78036, saving model to /content/drive/My Drive/Project_Pound/Xception.hdf5
Epoch 2/30
1/1 [==============================] - 1s 957ms/step - loss: 1.7394 - accuracy: 0.2043 - top_5_accuracy: 0.9130 - val_loss: 1.7175 - val_accuracy: 0.2692 - val_top_5_accuracy: 1.0000

Epoch 00002: val_loss did not improve from 1.78036
Epoch 3/30
1/1 [==============================] - 1s 912ms/step - loss: 1.6726 - accuracy: 0.2783 - top_5_accuracy: 0.9261 - val_loss: 1.6640 - val_accuracy: 0.2308 - val_top_5_accuracy: 1.0000

Epoch 00003: val_loss did not improve from 1.78036
Epoch 4/30
1/1 [==============================] - 1s 913ms/step - loss: 1.6147 - accuracy: 0.3130 - top_5_accuracy: 0.9435 - val_loss: 1.6200 - val_accuracy: 0.2308 - val_top_5_accuracy: 0.9615



In [ ]:
import plotly.graph_objects as go
fig = go.Figure()

fig.add_trace(go.Scatter(x=history.epoch,
                         y=history.history['accuracy'],
                         mode='lines+markers',
                         name='Training accuracy'))
fig.add_trace(go.Scatter(x=history.epoch,
                         y=history.history['val_accuracy'],
                         mode='lines+markers',
                         name='Validation accuracy'))
fig.update_layout(title='Accuracy',
                  xaxis=dict(title='Epoch'),
                  yaxis=dict(title='Percentage'))
fig.show()

In [ ]:
results = model.evaluate(val_x_scaled, val_images_y_encoded,batch_size=256)
print("test loss, test acc:", results)

1/1 [==============================] - 2s 2s/step - loss: 1.1009 - accuracy: 0.6211 - top_5_accuracy: 0.9609
test loss, test acc: [1.1008836030960083, 0.62109375, 0.9609375]


In [ ]:
results_train = model.evaluate(train_x_scaled, train_images_y_encoded,batch_size=256)
print("train loss, train acc:", results_train)

1/1 [==============================] - 1s 1s/step - loss: 0.8401 - accuracy: 0.7930 - top_5_accuracy: 0.9805
train loss, train acc: [0.8401279449462891, 0.79296875, 0.98046875]


# END